# Simple Usecase

We all copied this code at least once in our life:

```python
>>> X_train, X_test, y_train, y_test = train_test_split(
...     X, y, test_size=0.33, random_state=42)
```

and even if it's not that long, it's still a bit annoying to write/ copy. It must be a better way to do this. And there is! With the `Dataset` class, you can do this:



In [1]:
from skdataset.dataset import Dataset
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
import numpy as np
np.set_printoptions(threshold=10)

# make some data
X, y = make_classification(
    n_samples=200, 
    n_features=4, 
    n_classes=2, 
    n_clusters_per_class=1,
    random_state=42,
)

ds = Dataset(X=X, y=y)

NameError: name 'np' is not defined

In [ ]:
train_ds, test_ds = train_test_split(ds, test_size=0.33, random_state=42)

In [ ]:
len(train_ds) + len(test_ds) == len(X)

This by itself if not that impressive, but it's a good start. Let's see what else we can do with this class.
We also can manipulate the data in the dataset with `transform` method without changing the original data.

In [ ]:
def multiply_by_2(dataset: Dataset):
    dataset.X = dataset.X * 2
    return dataset

ds.transform(multiply_by_2)[:3, 'X'] 

In [ ]:
ds[:3, 'X']

or even filter some rows based on a condition and that filter will be applied to all the variables:


In [ ]:
import numpy as np

np.set_printoptions(threshold=10)

In [ ]:
ds.filter(lambda x: x.y == 1)

The abvious benefit of this is that you always have all of your data together, so you dont have to filter them once, and map the filter to the other parts(e.g. filtering some rows on X and then apply the same filter on y, or sample_weight)

## DatasetDict

Here we will interduce `DatasetDict` which is a dictionary-like object that holds multiple `Dataset` objects. It's very useful when you have multiple datasets(e.g. train, val and test) that you want to keep together.

You can create one by jast passing a dict with some keys and `Dataset` objects as values or call `split` with a spliter function on your dataset. Here is an example:

In [ ]:
from skdataset import DatasetDict

ds_dict = DatasetDict({'train': train_ds, 'test': test_ds})

ds_dict.keys()

In [ ]:

ds_dict = ds.split(train_test_split, test_size=0.33, random_state=42)

ds_dict.keys()

This new object has some handy attributes like `X_train` or `y_test` which all automatically generated from the keys in the dict:

In [ ]:
ds_dict.X_train

which is equal to saying `ds_dict['train']['X']`.

Imagine you have a function that you want to apply to all of your splits now, instead of looping over them, you can do `ds_dict.transform(func)` and it will apply the function to all of the splits and return a new `DatasetDict` object:

In [ ]:
ds_dict.transform(multiply_by_2)

## Put It All Together

Let's see how we can use all of these together. If we wanted to do it without `Dataset` It would be something like this:

```python
from skdataset.dataset import Dataset
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

X, y = make_classification(
    n_samples=200, 
    n_features=4, 
    n_classes=2, 
    n_clusters_per_class=1,
    random_state=42,
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42
)

model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)
print('Train score:', model.score(X_train, y_train))
print('Test score:', model.score(X_test, y_test))
```

In [ ]:
from skdataset.dataset import Dataset
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

X, y = make_classification(
    n_samples=200, 
    n_features=4, 
    n_classes=2, 
    n_clusters_per_class=1,
    random_state=42,
)

ds_dict = Dataset(X=X, y=y).split(train_test_split, test_size=0.33, random_state=42)

model = RandomForestClassifier(random_state=42)
model.fit(**ds_dict['train'])

print('Train score:', model.score(**ds_dict['train']))
print('Test score:', model.score(**ds_dict['test']))